# Converting Loan Amount Currency from Extracted Data

#### Extrated loan amount data has two parts: (1) a loan number and (2) a currency

Here, we convert all loans to USD using exchange date data from:

https://fxtop.com/en/historical-exchange-rates.php

Currencies are NOT inflation adjusted. Conversions are done an a yearly basis using average values for that year.

In [1]:
import sys
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

#### Define some relevant functions to extract the date in files

In [2]:
## define the function to extract the sector information and the date of loan agreement from the name of files
## file_name is a string, the function returns the list of strings describing sector and date
def extract_sector_date(file_name):
    sector= re.search(r'.*?--(\D*?)-(project|program|loan|credit){1}', file_name)
    date = re.search(r'(\d{4}_(january|february|march|april|may|june|july|august|september|october|november|december)_\d{1,2})_', file_name)
    sector_date =[]
    if sector:
            sector_date.append(sector.group(1))
    else:
            sector_date.append('error')
    if date:
            sector_date.append(date.group(1))
    else:
            sector_date.append('error')
    return sector_date

## This function is to extract the sector and date information for all files in a path
## Input is the name of the path, it is a string, 
## output is a dictionary with keys being filename and values being the list of sector and date
def extract_sector_date_files(path):
    dic = {}
    files = listdir_nohidden(path)
    for file_name in files:       
        r = extract_sector_date(file_name)
        flag = 'Success'
    
        if ('error' in r[1]):
            flag = 'Fail'
        dic[file_name] = [r, flag]
    return dic

## The listdir may return some unwanted hidden file. This function is to ignore those files
def listdir_nohidden(path):
    files = os.listdir(path)
    files.sort()
    for f in files:
        if not f.startswith('.'):
            yield f

## This function is to display the performance of extraction function 
def count_errors(dic):
    count = 0

    for key in dic.keys():
        if dic[key][1] == 'Fail':
            count +=1
            print(key)
            #print(dic[key][0])
        # else:
        #     print(dic[key][0])
    return count 

#### Read in our raw data

In [4]:
# Import both pymupdf and tesseract data files
raw_df = pd.read_csv('../Clean_Data/clean_data.csv')
raw_df_tess = pd.read_csv('../Clean_Data/clean_data_tess.csv')

# Drop the index file
raw_df = raw_df.drop(columns=raw_df.columns[0])
raw_df_tess = raw_df_tess.drop(columns=raw_df_tess.columns[0])

# Combine into a single dataframe
df = pd.concat([raw_df, raw_df_tess])

# Extract year of loan
df['year'] = df.filename.apply(lambda x: extract_sector_date(x)[1][0:4])
df.set_index(keys='filename',inplace=True) # Make the index the filename
df_loan = pd.read_csv('../Clean_Data/loan_data.csv')
df_loan.set_index(keys='filename',inplace=True)

# Import our classification for the country for the loans
pymu_max = pd.read_csv('../country_data/PyMu_max_countries.csv')
tess_max = pd.read_csv('../country_data/tess_max_countries.csv')
df_max = pd.concat([pymu_max, tess_max])
df_max.set_index(keys='proj_name',inplace=True)
df_max = df_max.drop(columns='countries')

# Combine all of these dataframes
df_total = pd.concat([df,df_loan,df_max], axis=1)

# We will simply drop the null values for the loans and countries since they're not going to be relevant to us
df_total = df_total.dropna()

# Convert strings to numbers
df_total['year'] = pd.to_numeric(df_total['year'])
df_total['loan_amount'] = pd.to_numeric(df_total['loan_amount'])
df_total.drop(columns=['raw_text', 'num_char', 'clean_text']).to_csv('../country_data/Collected_data.csv')

#### Determine which currency types are present:

In [6]:
df_total.currency.value_counts().index

Index(['$', 'EUR', 'DEM', 'SDR', 'FRF', 'JPY', 'GBP', 'INR', 'SEK'], dtype='object')

#### Consolidates all currency conversions into a dataframe called df_curr

In [9]:
df_test = df_total.copy()
df_test['new_loan'] = df_test.loan_amount
column_names = ["currency", "year", "average"]
df_curr = pd.DataFrame(columns = column_names)

for curr in df_test.currency.value_counts().index:
    if curr != '$':
        temp_df = pd.read_csv('../country_data/' + curr + '_USD.csv')
        temp_df = temp_df.rename(columns={"CURRENCY": "currency", "YEAR": "year", "AVERAGE": "average"})
        temp_df["currency"] = curr
        temp_df = temp_df[["currency", "year", "average"]]
        df_curr = pd.concat([df_curr, temp_df])
        
df_curr

,currency,year,average
0,EUR,1990,1.271003
1,EUR,1991,1.236353
2,EUR,1992,1.301715
3,EUR,1993,1.184672
4,EUR,1994,1.201994
...,...,...,...
25,SEK,2015,0.118619
26,SEK,2016,0.116982
27,SEK,2017,0.117242
28,SEK,2018,0.115209


In [10]:
# Check dataframe
df_test

,raw_text,num_char,clean_text,year,loan_amount,currency,max_country,world_region,new_loan
1990_april_24_587321468019152780_conformed-copy--l3186--forestry-sector-project--loan-agreement.txt,CONFORMED COPY\n\nLOAN NUMBER 3186 IVC\n\nLoan...,41383.0,"['conformed', 'copy', 'loan', 'number', 'ivc',...",1990,80000000,$,united states,northern america,80000000
1990_april_24_668811468165272290_conformed-copy--c2120--water-supply-project--loan-agreement.txt,...,16922.0,"['conformed', 'copy', 'loan', 'number', 'slu',...",1990,2500000,$,united states,northern america,2500000
1990_april_25_904191468298750561_conformed-copy--l3190--environment-management-project--loan-agreement.txt,...,33100.0,"['conformed', 'copy', 'loan', 'number', 'pol',...",1990,18000000,$,poland,eastern europe,18000000
1990_april_30_410811468040573756_conformed-copy--l3180--rural-electrification-project--loan-agreement.txt,CONF...,41030.0,"['conformed', 'copy', 'loan', 'number', 'ind',...",1990,329000000,$,indonesia,asia (ex. near east),329000000
1990_april_30_725911468042268845_conformed-copy--l3182--third-telecommunications-project--loan-agreement.txt,CONFO...,33593.0,"['conformed', 'copy', 'loan', 'number', 'ind',...",1990,350000000,$,peru,latin amer. & carib,350000000
...,...,...,...,...,...,...,...,...,...
2019_october_24_126261574264006073_official-documents-additional-financing-agreement-for-loan-9008-ar-closing-package.txt,Public Disclosure Authorized\n\n\n\nPublic Dis...,64455.0,"['public', 'disclosure', 'authorized', 'public...",2019,245000000,$,united states,northern america,245000000
2019_october_24_342391574201578264_official-documents-loan-agreement-for-loan-9007-ar-closing-package.txt,Public Disclosure Authorized\n\n\n\nPublic Dis...,35989.0,"['public', 'disclosure', 'authorized', 'public...",2019,150000000,$,united states,northern america,150000000
2019_september_13_710891569417913880_official-documents-loan-agreement-for-loan-8798-co-closing-package.txt,Public Disclosure Authorized\n\n\n\nPublic Dis...,33116.0,"['public', 'disclosure', 'authorized', 'public...",2019,2500000,$,colombia,latin amer. & carib,2500000
2019_september_23_867961572361092133_official-documents-loan-agreement-for-loan-9005-ro-closing-package.txt,Public Disclosure Authorized\n\n\n\nPublic Dis...,31004.0,"['public', 'disclosure', 'authorized', 'public...",2019,25000000,EUR,oman,near east,25000000


#### Merge dataframes together to have our data converted to a common loan amount format

This is done by merging the conversion dataframe with our original dataframe. Then we can multiply the columns using the currency conversion rates. Since our null values will now all be ones in USD, we convert the null values to 1 since that results to multiplying by 1

In [11]:
merged = pd.merge(df_test, df_curr, on = ["currency", "year"], how = "left").set_index(df_test.index)
merged['average'] = merged['average'].fillna(1)
merged['new_loan_amt'] = merged['loan_amount']*merged['average']
merged.loan_amount = merged.new_loan_amt
merged.drop(labels=['currency', 'new_loan', 'average', 'new_loan_amt', 'raw_text', 'num_char', 'clean_text'], axis=1, inplace=True)
merged

,year,loan_amount,max_country,world_region
1990_april_24_587321468019152780_conformed-copy--l3186--forestry-sector-project--loan-agreement.txt,1990,8.000000e+07,united states,northern america
1990_april_24_668811468165272290_conformed-copy--c2120--water-supply-project--loan-agreement.txt,1990,2.500000e+06,united states,northern america
1990_april_25_904191468298750561_conformed-copy--l3190--environment-management-project--loan-agreement.txt,1990,1.800000e+07,poland,eastern europe
1990_april_30_410811468040573756_conformed-copy--l3180--rural-electrification-project--loan-agreement.txt,1990,3.290000e+08,indonesia,asia (ex. near east)
1990_april_30_725911468042268845_conformed-copy--l3182--third-telecommunications-project--loan-agreement.txt,1990,3.500000e+08,peru,latin amer. & carib
...,...,...,...,...
2019_october_24_126261574264006073_official-documents-additional-financing-agreement-for-loan-9008-ar-closing-package.txt,2019,2.450000e+08,united states,northern america
2019_october_24_342391574201578264_official-documents-loan-agreement-for-loan-9007-ar-closing-package.txt,2019,1.500000e+08,united states,northern america
2019_september_13_710891569417913880_official-documents-loan-agreement-for-loan-8798-co-closing-package.txt,2019,2.500000e+06,colombia,latin amer. & carib
2019_september_23_867961572361092133_official-documents-loan-agreement-for-loan-9005-ro-closing-package.txt,2019,2.798686e+07,oman,near east


#### Export data to our Finalized loan amount csv file

In [ ]:
merged.to_csv('Final_loan_amount.csv')

#### Old code. Ignore

In [7]:
# df_test = df_total.copy()
# df_test['new_loan'] = df_total.copy()

# for curr in df_test.currency.value_counts().index:
#     if curr != '$':
#         temp_df = pd.read_csv('../country_data/' + curr + '_USD.csv')
#         for yr in temp_df.YEAR:
#             if len(df_test[((df_total.currency==curr) & (df_test.year==yr))])!=0:
#                 print(i, yr)
#                 df_test[(df_test.currency==curr) & (df_test.year==yr)]['new_loan'] = df_test[(df_test.currency==curr) & (df_test.year==yr)].loan_amount * temp_df[temp_df['YEAR']==yr].AVERAGE.values
# #                 df_total[(df_total.currency==i) & (df_total.year==yr)].currency.replace(to_replace=i, value='$', inplace=True)

ValueError: Wrong number of items passed 8, placement implies 1